# Cleaning Kangaroo dataset - deprecated : use data_preparator.py instead

In [37]:
import pandas as pd

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

from peb_util import map_label_to_kwh
from outlier_util import remove_outliers_iqr

In [38]:
df = pd.read_csv('./data/Kangaroo.csv')

### Drop duplicate rows

In [39]:
df.drop_duplicates('id', inplace=True)

Drop useless columns

In [40]:
df.drop(columns=['Unnamed: 0', 'url', 'type', 'locality', 'monthlyCost', 'hasBalcony', 'accessibleDisabledPeople', 'kitchenSurface', 'hasTerrace', 'hasGarden', 'gardenOrientation', 'roomCount', 'streetFacadeWidth', 'floorCount', 'floodZoneType', 'terraceOrientation', 'hasAttic', 'hasBasement', 'diningRoomSurface', 'hasDiningRoom', 'hasLift', 'heatingType', 'hasLivingRoom', 'livingRoomSurface', 'gardenSurface', 'parkingCountIndoor', 'hasAirConditioning', 'hasArmoredDoor', 'hasVisiophone', 'bathroomCount'], inplace=True)

Replace values

In [41]:

df['hasOffice'].replace(pd.NA, False, inplace=True)
df['hasPhotovoltaicPanels'].replace(pd.NA, False, inplace=True)
df['hasHeatPump'].replace(pd.NA, False, inplace=True)
df['hasThermicPanels'].replace(pd.NA, False, inplace=True)
df['hasFireplace'].replace(pd.NA, False, inplace=True)
df['hasDressingRoom'].replace(pd.NA, False, inplace=True)
df['hasSwimmingPool'].replace(pd.NA, False, inplace=True)

df['terraceSurface'].fillna(0.0, inplace=True)
df['parkingCountOutdoor'].fillna(0, inplace=True)
df['landSurface'].fillna(0.0, inplace=True)

Replace True_False with 1_0

In [42]:
# Convert boolean columns to integers (True -> 1, False -> 0)
bool_columns = df.select_dtypes(include='bool').columns
for column in bool_columns:
    df[column] = df[column].astype(int)

# Verify the conversion
print(f"Converted {len(bool_columns)} boolean columns to integers:")
print(bool_columns.tolist())
print("\nDataFrame dtypes after conversion:")
print(df.dtypes.value_counts())

Converted 7 boolean columns to integers:
['hasDressingRoom', 'hasHeatPump', 'hasPhotovoltaicPanels', 'hasThermicPanels', 'hasOffice', 'hasSwimmingPool', 'hasFireplace']

DataFrame dtypes after conversion:
float64    10
int64       8
object      5
Name: count, dtype: int64


Strip & Uppercase strings

In [43]:
string_columns = df.select_dtypes(include='object').columns

for column in string_columns:
    # Apply strip and uppercase, but only to non-NaN string values
    df[column] = df[column].apply(lambda x: x.strip().upper() if isinstance(x, str) else x)
    
    # Count the modified values
    modified_count = df[column].notna().sum()
    
    # Print the result
    print(f"Processed column '{column}': {modified_count} values modified")

Processed column 'subtype': 80367 values modified
Processed column 'province': 80367 values modified
Processed column 'buildingCondition': 57991 values modified
Processed column 'kitchenType': 41916 values modified
Processed column 'epcScore': 65390 values modified


Convert PEB before removing province col

In [44]:
df["epc_kwh"] = df.apply(map_label_to_kwh, axis=1)
df.drop(columns=['epcScore', 'province'], inplace=True)

Remove Outliers first

In [45]:
columns_for_outlier_removal = [
    'price',
    'habitableSurface'
]

min_thresholds = { 'habitableSurface' : 25 }  

df = remove_outliers_iqr(df, columns_for_outlier_removal, multiplier=1.5, min_thresholds=min_thresholds)


### Imputing values after removing outliers

In [46]:
df['facedeCount'].fillna(int(df['facedeCount'].median()), inplace=True)
df['buildingConstructionYear'].fillna(int(df['buildingConstructionYear'].median()), inplace=True)
df['habitableSurface'].fillna(int(df['habitableSurface'].median()), inplace=True)
df['bedroomCount'].fillna(int(df['bedroomCount'].median()), inplace=True)
df['epc_kwh'].fillna(int(df['epc_kwh'].median()), inplace=True)

Clamp facade count to the range 1-4

In [47]:
# # Clamp facedeCount to the range 1-4
# print(f"Rows with facedeCount > 4 before clamping: {sum(df['facedeCount'] > 4)}")
# df['facedeCount'] = df['facedeCount'].clip(lower=1, upper=4)
# print(f"Max facedeCount after clamping: {df['facedeCount'].max()}")
# print(f"Min facedeCount after clamping: {df['facedeCount'].min()}")

### Converting values

Find trend and convert Building Condition according to it

In [48]:
building_condition_price = df.groupby('buildingCondition')['price'].mean().round().sort_values(ascending=False)
df['buildingCondition'] = df['buildingCondition'].map(building_condition_price.to_dict())

Find trend for postalCode

In [49]:
# Calculate the average price/m2 per postalCode
df['price_per_m2'] = df['price'] / df['habitableSurface']

postal_code_price = df.groupby('postCode')['price_per_m2'].mean().round().sort_values(ascending=False)

df['postCode'] = df['postCode'].map(postal_code_price.to_dict())
df.drop(columns=['price_per_m2'], inplace=True)


Find trend and convert SubType according to it

In [50]:
subtype_price = df.groupby('subtype')['price'].mean().round().sort_values(ascending=False)
df['subtype'] = df['subtype'].map(subtype_price.to_dict())

Find trend and convert KitchenType according to it

In [51]:
kitchen_type_price = df.groupby('kitchenType')['price'].mean().round().sort_values(ascending=False)
df['kitchenType'] = df['kitchenType'].map(kitchen_type_price.to_dict())

Replace by mode

In [52]:
df['buildingCondition'].fillna(df['buildingCondition'].mode()[0], inplace=True)
df['subtype'].fillna(df['subtype'].mode()[0], inplace=True)
df['kitchenType'].fillna(df['kitchenType'].mode()[0], inplace=True)

Drop type & dropna

In [53]:
df.dropna(inplace=True)


for col in df.columns:
    df[col] = df[col].astype(int)

price_corr = df.corr()['price'].sort_values(ascending=False)
print("Correlation with price:")
print(price_corr)

Correlation with price:
price                       1.000000
habitableSurface            0.522432
toiletCount                 0.441864
bedroomCount                0.382868
postCode                    0.371228
subtype                     0.338972
kitchenType                 0.282991
buildingCondition           0.251002
hasPhotovoltaicPanels       0.208022
hasHeatPump                 0.186513
facedeCount                 0.183534
landSurface                 0.180497
hasOffice                   0.171144
terraceSurface              0.128613
hasDressingRoom             0.128157
parkingCountOutdoor         0.126360
hasFireplace                0.125721
buildingConstructionYear    0.125469
hasThermicPanels            0.118150
hasSwimmingPool             0.114555
id                          0.020205
epc_kwh                    -0.226562
Name: price, dtype: float64


### Finally save the cleaned dataset

In [54]:
# Save the cleaned dataset to CSV 
df.to_csv('./data/kangaroo-cleaned-with-id.csv', index=False)
print(f"Dataset saved with {len(df)} rows and {len(df.columns)} columns")

Dataset saved with 44809 rows and 22 columns
